# Regras Regex dos Atos

In [21]:
import os
import sys
module_path = os.path.abspath(os.path.join('../regex'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Importações

In [22]:
import helper as helper
from atos.aposentadoria import Retirements
from atos.reversoes import Revertions
from atos.nomeacao import NomeacaoComissionados
from atos.exoneracao import Exoneracao


### Diretórios

In [23]:
dodfs_test = "../data/test/"
dodfs_test_files = helper.get_txts(dodfs_test)
output = "../results"

../data/test/


## Classes

In [24]:
ret = Retirements(dodfs_test_files[0])
helper._build_act_txt(ret.acts_str, ret.name, output)
helper.print_dataframe(ret.data_frame)

,Tipo do Ato,SEI,Nome,Matrícula,Tipo de Aposentadoria,Cargo,Classe,Padrao,Quadro,Fundamento Legal,Orgao,Vigencia,Matricula SIAPE
0,Aposentadoria,,nan,31.275-4,nan,Professor de Educacao Basica,nan,25,Quadro de Pessoal do Distrito Federal,"artigo 40, 1o, inciso I, in fine, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 6o-A da Emenda Constitucional no 41, de 31 de dezembro de 2003, incluido pela Emenda Constitucional no 70, de 29 de marco de 2012, e o artigo 18",nan,nan,nan
1,Aposentadoria,00040-00026111/2019-15,nan,204.741-1,nan,Professor de Educacao Basica,nan,19,Quadro de Pessoal do Distrito Federal,"artigo 40, 1o, inciso I, in fine, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 6o-A da Emenda Constitucional no 41, de 31 de dezembro de 2003, incluido pela Emenda Constitucional no 70, de 29 de marco de 2012, e o artigo 18",nan,nan,nan
2,Aposentadoria,00040-00032116/2019-87,nan,20.820-5,nan,Agente de Gestao Educacional/Conservacao e Limpeza,nan,2,Quadro de Pessoal do Distrito Federal,nan,nan,nan,nan
3,Aposentadoria,00040-00028368/2019-10,nan,48.117-3,nan,Agente de Gestao Educacional/Vigilancia,nan,1,Quadro de Pessoal do Distrito Federal,nan,nan,nan,nan
4,Aposentadoria,00040-00032095/2019-08,nan,32.527-9,nan,Professor de Educacao Basica,nan,25,Quadro de Pessoal do Distrito Federal,nan,nan,nan,nan
5,Aposentadoria,00040-00030212/2019-91,nan,21.470-1,nan,Agente de Gestao Educacional/Servico Gerais,nan,3,Quadro de Pessoal do Distrito Federal,nan,nan,nan,nan
6,Aposentadoria,00040-00029585/2019-19,nan,41.095-0,nan,Agente de Gestao Educacional/Conservacao e Limpeza,nan,3,Quadro de Pessoal do Distrito Federal,nan,nan,nan,nan
7,Aposentadoria,00040-00028387/2019-38,nan,67.642-X,nan,Agente de Gestao Educacional/Servicos Gerais,nan,1,Quadro de Pessoal do Distrito Federal,nan,nan,nan,nan
8,Aposentadoria,00040-00034071/2019-85,nan,20.852-3,nan,Agente de Gestao Educacional/Conservacao e Limpeza,nan,3,Quadro de Pessoal do Distrito Federal,nan,nan,nan,nan
9,Aposentadoria,00040-00036048/2019-25,nan,25.155-0,nan,Agente de Gestao Educacional/Portaria,nan,2,Quadro de Pessoal do Distrito Federal,nan,nan,nan,nan


In [25]:
rev = Revertions(dodfs_test_files[4])
helper._build_act_txt(rev.acts_str, rev.name, output)
helper.print_dataframe(rev.data_frame)

,Tipo do Ato,SEI,Nome,Matrícula,Cargo,Classe,Padrao,Quadro,Fundamento Legal,Orgao,Vigencia,Matricula SIAPE
0,Reversão,080.001995/2006,a servidora NILVA DE JESUS MEIRELLES,32.299-7,Professor,A,nan,Quadro de Pessoal do Distrito Federal,"artigo 34, inciso II, da Lei Complementar no 840, de 26 de dezembro de 2011",nan,nan,nan


## Exoeração

In [26]:
exo = Exoneracao(dodfs_test_files[0])
helper._build_act_txt(exo.acts_str, exo.name, output)
helper.print_dataframe(exo.data_frame)

,tipo,nome,matricula,simbolo,cargo_comissao,lotacao,orgao,vigencia,pedido,cargo_efetivo,siape,motivo
0,Exoneração,EDUARDO PIRES FERNANDES,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,Exoneração,ELAENE CRISTINA DA SILVA MENDES,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,Exoneração,SAMARA NERY DE OLIVEIRA ALMEIDA,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,Exoneração,AGENOR VIEIRA DOS SANTOS JUNIOR,nan,nan,nan,nan,nan,nan,a pedido,nan,nan,nan
4,Exoneração,MARIANA ALMADA VIANA,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,Exoneração,DANIELA GOMES DO NASCIMENTO,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6,Exoneração,GLACILENE CAIANA DA SILVA MACEDO,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7,Exoneração,FLAVIO CIRENO FERNANDES,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8,Exoneração,JOSIANE DALLASTRA,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
9,Exoneração,ELIDA PEREIRA DOS SANTOS,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


## Nomeação

In [27]:
nom = NomeacaoComissionados(dodfs_test_files[0])
helper._build_act_txt(nom.acts_str, nom.name, output)
helper.print_dataframe(nom.data_frame)

,tipo,nome,cargo_efetivo,matricula,siape,simbolo,cargo_comissao,lotacao,orgao
0,Nomeação,FABIO DIAS GALVAO,nan,nan,nan,nan,nan,nan,nan
1,Nomeação,ELAENE CRISTINA DA SILVA MENDES,nan,nan,nan,DFG-14,nan,nan,nan
2,Nomeação,SAVIA COIMBRA SANTOS,nan,nan,nan,nan,nan,nan,nan
3,Nomeação,KELLY CRISTINA SIMOES GOMES,nan,nan,nan,DFA-,nan,nan,nan
4,Nomeação,MARILDA ROSA COELHO,nan,nan,nan,nan,nan,nan,nan
5,Nomeação,VITORIA REGIA DE OLIVEIRA PIRES,nan,nan,nan,nan,nan,nan,nan
6,Nomeação,HUGO PIRES SOUSA,nan,nan,nan,nan,nan,nan,nan
7,Nomeação,JOAO MARCELO PACHECO,nan,nan,nan,nan,nan,nan,nan
8,Nomeação,MARIANA ALMADA VIANA,nan,nan,nan,nan,nan,nan,nan
9,Nomeação,RONI IVAN ROCHA DE OLIVEIRA,nan,nan,nan,nan,nan,nan,nan


## Abono de Permanência

In [28]:
class stayAllowance(Regex):
    
    def __init__(self, text):
        super().__init__(text)
        self._columns = ["Nome do Servidor", "Matrícula", "Cargo Efetivo", "Classe", 
                         "Padrão", "Quadro pessoal permanente ou Suplementar",
                         "Fundamento Legal do abono de permanência", "Órgão" 
                         "Processo GDF/SEI", "Vigencia","Matricula SIAPE"]
        
        self.rules = {"nome": "\s([^,]*?),\smatricula",
                      "matricula":"matricula\s?n?o?\s([\s\S]*?)[,|\s]",
                      "cargo": "Cargo\s[d|D]?[e|E]?\s([\s\S]*?),",
                      "classe": "[C|c]lasse\s([\s\S]*?),",
                      "padrao": "[p|P]adr[a|ã]o\s([\s\S]*?),",
                      "quadro": "d?[e|a|o]?(Quadro[\s\S]*?)[,|;|.]",
                      "fundamento": "nos\stermos\sdo\s([\s\S]*?),\sa?\s",
                      "orgao": "Lotacao: ([\s\S]*?)[.]",
                      "processo": "Processo SEI: [\s\S]*?\.\n",
                      "vigencia": "a contar de [\s\S]*?\,",
                      "siape": "[S|s][I|i][A|a][P|p][E|e]\s[N|n]?[o|O]?\s([\s\S]*?)[,| | .]"}
                      
        self._raw_acts = self._extract_instances()   
        #self._acts = self._acts_props()
        #self.data_frame = self._build_dataframe()
        
        
    
    def _act_props(self, sei, act_raw):
        act = {}
        act["tipo_ato"] = "Abono Permência"
        act["sei"] = sei
        for key in self.rules:
            try:
                act[key], = self.find_in_act(self.rules[key], act_raw)
            except:
                act[key] = "nan"

        return act
    
    def _acts_props(self):
        acts = []
        for sei, raw in self._raw_acts.items():
            act = self._act_props(sei, raw)
            acts.append(act)
        return acts        
        
    
    def _extract_instances(self):
        start = "(Abono DE PERMANENCIA (ao|equiva)[\s\S]*?)\s"
        body = "([\s\S]*?)"
        end = "(\.\n)"
        rule = start + body + end
        rule = re.compile("(Abono DE PERMANENCIA (ao|equiva)[\s\S]*?\.\n)", re.IGNORECASE)
        found = self.find_all(rule)
        results = {}
        for instance in found:
            start, body, sei = instance
            results[sei] = body
            
        return results

NameError: name 'Regex' is not defined

## Retificações - (Aposentadorias)

In [29]:
class ratificationsRetirements(Regex):
    
    def __init__(self, text):
        super().__init__(text)
        self._columns = ["Tipo de Documento", "Número do documento", "Data do documento ",
                         "Número do DODF", "Data do DODF", "Página do DODF", "Nome do Servidor",
                         "Matrícula", "Cargo", "Classe", "Padrao", "Matricula SIAPE",
                         "Informação Errada", "Informação Corrigida"]
        
        self.rules = {"Tipo doc": "",
                      "Num doc": "",
                      "Data doc": "",
                      "Num dodf": "",
                      "Data dodf": "",
                      "Pag dodf": "",
                      "nome": "\s([^,]*?),\smatricula",
                      "matricula":"matricula\s?n?o?\s([\s\S]*?)[,|\s]",
                      "cargo": "Cargo\s[d|D]?[e|E]?\s([\s\S]*?),",
                      "classe": "[C|c]lasse\s([\s\S]*?),",
                      "padrao": "[p|P]adr[a|ã]o\s([\s\S]*?),",
                      "siape": "[S|s][I|i][A|a][P|p][E|e]\s[N|n]?[o|O]?\s([\s\S]*?)[,| | .]",
                      "Info errada": "",
                      "Info corrigida": ""
                     }
                      
        self._raw_acts = self._extract_instances()   
        #self._acts = self._acts_props()
        #self.data_frame = self._build_dataframe()
        
        
    
    def _act_props(self, sei, act_raw):
        act = {}
        act["tipo_ato"] = "Retificações-(Aposentadorias)"
        act["sei"] = sei
        for key in self.rules:
            try:
                act[key], = self.find_in_act(self.rules[key], act_raw)
            except:
                act[key] = "nan"

        return act
    
    def _acts_props(self):
        acts = []
        for sei, raw in self._raw_acts.items():
            act = self._act_props(sei, raw)
            acts.append(act)
        return acts        
        
    
    def _extract_instances(self):
        start = "(RETIFICAR,.*?ato\sque\sconcedeu\saposentadoria[\s\S]*?)"
        body = ""
        end = "(\.\n)"
        rule = start + body + end
        #rule = re.compile("(RETIFICAR,.*?ato\sque\sconcedeu\saposentadoria[\s\S]*?\.\n)", re.IGNORECASE)
        found = self.find_all(rule)
        results = {}
        for instance in found:
            *start, body, sei = instance
            results[sei] = body
        
        print(results)
        return results

NameError: name 'Regex' is not defined

## Substituição de Funções

In [30]:
class roleReplacement(Regex):
    
    def __init__(self, text):
        super().__init__(text)
        self._columns = ["Nome do Servidor Substituto", "Matrícula do Servidor Substituto", 
                         "Nome do Servidor a ser Substituido", "Matrícula do Servidor a ser Substituido"
                         "Cargo", "Símbolo do cargo do servidor substituto",
                         "Cargo comissionado objeto da substituição",
                         "Símbolo do cargo comissionado objeto da substituição",
                         "Hierarquia da Lotação", "Órgão", "Data Inicial da Vigência", 
                         "Data Final de Vigência", "Matrícula SIAPE", "Motivo"]
        
        self.rules = {"Nome Serv Substituto": "",
                      "Matricula Serv Substituto": "",
                      "Nome do Servidor a ser Substituido": "",
                      "Matrícula do Servidor a ser Substituido": "",
                      "cargo": "Cargo\s[d|D]?[e|E]?\s([\s\S]*?),",
                      "Símbolo do cargo do servidor substituto": "",
                      "Cargo comissionado objeto da substituição": "",
                      "Hierarquia da Lotação": "",
                      "orgao": "Lotacao: ([\s\S]*?)[.]",
                      "Data Inicial da Vigência": "",
                      "Data Final de Vigência": "",
                      "siape": "[S|s][I|i][A|a][P|p][E|e]\s[N|n]?[o|O]?\s([\s\S]*?)[,| | .]",
                      "Motivo": ""
                     }
                      
        self._raw_acts = self._extract_instances()   
        #self._acts = self._acts_props()
        #self.data_frame = self._build_dataframe()
        
        
    
    def _act_props(self, sei, act_raw):
        act = {}
        act["tipo_ato"] = "Substituição de Funções"
        act["sei"] = sei
        for key in self.rules:
            try:
                act[key], = self.find_in_act(self.rules[key], act_raw)
            except:
                act[key] = "nan"

        return act
    
    def _acts_props(self):
        acts = []
        for sei, raw in self._raw_acts.items():
            act = self._act_props(sei, raw)
            acts.append(act)
        return acts        
        
    
    def _extract_instances(self):
        start = "(DESIGNAR)"
        body = "(.*?para\ssubstituir[\s\S]*?)"
        end = "(\.\n)"
        rule = start + body + end
        found = self.find_all(rule)
        results = {}
        for instance in found:
            start, body, sei = instance
            results[sei] = body
        
        print(results)
        return results

NameError: name 'Regex' is not defined